## **Importing all the important libraries**

In [ ]:
#importing all the important libraries

from __future__ import print_function, division

import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
from torchvision import datasets,models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import time
import copy

import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.ion()
%matplotlib inline

**labeling the cloth classes**

In [ ]:
class_label = {'2':'Blazer', '3':'Blouse'}


# **Transforming the dataset**

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),                                                 # transform ‘to_tensor’ will be used to convert the PIL image to a PyTorch tensor (multidimensional array)
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# **Importing the pretrained ResNet Model**

In [ ]:
model_ts = torch.load('trained_model.pt')
model_ts.eval()
model_ts

**Building a activation function to get the output of first fully connected layer**

In [ ]:
# building the function to get the output of first fully connected layer
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


**Define a function to get a path of each input image**

In [ ]:
# Define a function for getting a path of each input image


class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
# Getting the first fully connected layer for the each and every image in train dataset and store the data into df_train dataframe

data_dir_train = '/content/drive/My Drive/Colab Notebooks/MVC/data/train'                      # path of image directory
dataset_train = ImageFolderWithPaths(data_dir_train, transform=data_transforms['train'])        # our custom dataset

dataloader_train = torch.utils.data.DataLoader(dataset_train)   

# generate a empty dataset
df_train = pd.DataFrame()

# iterating over a train dataset

for inputs, label, img_path in dataloader_train:
      model_ts.avgpool.register_forward_hook(get_activation('name'))              #  first fully connected layers output
      input_img = inputs                                                   # input image of size [1, 3, 224, 224]
      output = model_ts(input_img)                                                      # fit the image to above model and get output of first fully connected layer of that image
      output_FC = np.array(activation['name'])                                                    # convert above output to list
      output_FC = output_FC.reshape(1,2048)                       
      label = img_path[0].split('/')[-2]                                                # getting a category label from image path
      data = {"category":label,"img_path":img_path[0], "img_vec":output_FC}             # fitting the above data to the empty datafrme
      df_train = df_train.append(data, ignore_index=True) 

  
df_train.head()

In [ ]:
# Getting the first fully connected layer for the each and every image in valid dataset and store the data into df_valid dataframe

data_dir_valid = 'data/valid'                      # path of image directory
dataset_valid = ImageFolderWithPaths(data_dir_valid, transform=data_transforms['valid'])        # our custom dataset

dataloader_valid = torch.utils.data.DataLoader(dataset_valid)                                   


# generate a empty dataset
df_valid = pd.DataFrame()


# iterating over a valid dataset

for input, label, img_path in dataloader_valid:
      model_ts.avgpool.register_forward_hook(get_activation('name'))              #  first fully connected layers output
      input_img = input#.to(device)                                                     # input image of size [1, 3, 224, 224]
      output = model_ts(input_img)                                                      # fit the image to above model and get output of first fully connected layer of that image
      output_FC = np.array(activation['name'])                                                   # convert above output to list                        
      output_FC = output_FC.reshape(1,2048)
      label = img_path[0].split('/')[-2]                                                # getting a category label from image path
      data = {"category":label,"img_path":img_path[0], "img_vec":output_FC}             # fitting the above data to the empty datafrme
      df_valid = df_valid.append(data, ignore_index=True) 


df_valid.head()

In [ ]:
# Getting the first fully connected layer for the each and every image in train dataset and store the data into df_test dataframe

data_dir_test = 'data/test'                      # path of image directory
dataset_test = ImageFolderWithPaths(data_dir_test, transform=data_transforms['test'])        # our custom dataset

dataloader_test = torch.utils.data.DataLoader(dataset_test)   

# generate a empty dataset
df_test = pd.DataFrame()

# iterating over a train dataset

for input, label, img_path in dataloader_test:
      model_ts.avgpool.register_forward_hook(get_activation('name'))              #  first fully connected layers output
      input_img = input#.to(device)                                                     # input image of size [1, 3, 224, 224]
      output = model_ts(input_img)                                                      # fit the image to above model and get output of first fully connected layer of that image
      output_FC = np.array(activation['name'])                                                    # convert above output to list
      output_FC = output_FC.reshape(1,2048)                       
      label = img_path[0].split('/')[-2]                                                # getting a category label from image path
      data = {"category":label,"img_path":img_path[0], "img_vec":output_FC}             # fitting the above data to the empty datafrme
      df_test = df_test.append(data, ignore_index=True) 

  
df_test.head()

In [ ]:
# downloading a dataframes
from google.colab import files

df_train.to_csv('df_train.csv') 


df_valid.to_csv('df_valid.csv') 


df_test.to_csv('df_test.csv') 


**Importing a attributes dataframe**

In [ ]:
path = "attributes1.csv"  # paste the path attributes1.csv file
#path = "/content/drive/My Drive/Colab Notebooks/MVC/data/attributes1.csv"
attributes1 = pd.read_csv(path)
attributes1.info()

In [ ]:
attributes = attributes1.drop(['Unnamed: 0','Unnamed: 0.1','image_name','sold'], axis = 1)         # dropping the Image_name column
attributes = attributes.rename_axis('attributes_vec').values   # converting the dataframe to the numpy array
attributes[:5]

**choosing the image on basis of attributes**

In [ ]:
def choose_image(attributes_vector):
    att_sim = []
    for i in range(len(attributes)):
        cos_simm = np.dot(attributes_vector,attributes[i]) / (np.linalg.norm(attributes_vector) * np.linalg.norm(attributes[i]))   # caluclating cosine similarity between attibutes vec
        att_sim.append(cos_simm)
    att_sim = np.array(att_sim)                                            # convert list ot numpy arraay
    
    list1 = np.where(att_sim == np.amax(att_sim))                  # getting a indices of all the maximum  values from att
    list1 = list1[0]

    add = 0
    for i in range(len(list1)):
        a = attributes1["sold"][i]
        add += a                                                    # adding entries of sold column on the basis of indices from list1

    p = []
    for i in range(len(list1)):
        p1 = attributes1["sold"][i] / add
        p.append(p1)                                                 # getting a probabilty of each entry of sold column on the basis of indices from list1 
    
    #p = p.to_numpy()

    list2 = np.random.choice(list1, 6, p=p, replace=False)                       # getting a list of indices(top 6) on the basis of probability p randomly

    fig = plt.figure(figsize=(10,10))
    for j in range(len(list2)):
        i = list2[j]
        categoty_label = df_test["category"][i]       # get a category_label from index       
        category_name = class_label[categoty_label]    # get a category from category_label using class_label dictionary
        img = mpimg.imread(df_test["img_path"][i])    # reading a image using mpimg
        ax = fig.add_subplot(2,3,j+1)
        ax.imshow(img)
        title = str(j) + " " + category_name
        plt.title(title)

    return list2


**Recommending the images**

In [ ]:
import warnings

loader = transforms.Compose([transforms.RandomResizedCrop(224), transforms.ToTensor()])   # passing the input image through transforms

def recommended_images(number):
    #list = np.random.choice(list1, 6, p=p)                       # getting a list of indices on the basis of probability p
    a = list2[number]                                                      # getting a index of above choosen image

    attributes1["sold"][a] = attributes1["sold"][a] + 1                  # adding 1 to the sold column each time when customer chooses that image

    image_name = df_test["img_path"][a]                                  # getting a path of that image from above index


    # Using PyTorch Cosine Similarity
    cos_sim = []
    for i in range(len(df_train["category"])):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        vec1 = torch.as_tensor(df_test["img_vec"][a])                      # # getting a vec of that image from above index from df_test dataframe
        vec1.requires_grad = True
        vec2 = torch.as_tensor(df_train["img_vec"][i])  
        vec2.requires_grad = True           
        cos_similarity = cos(vec1, vec2)             # calculate the cos angle between output_FC and train img_vec
        cos_similarity = cos_similarity.item()                       
        cos_sim.append(cos_similarity)

    cos_sim = np.array(cos_sim)                                             # convert list ot numpy arraay

    
    
    # run through the list 
    list3 = cos_sim.argsort()[-6:][::-1]               # top 6 maximum value of cos_similarity list
    
    # showing reference image
    img1 = mpimg.imread(image_name)
    plt.figure(figsize=(7,7))
    cat_label = df_test["category"][a]       # get a category_label from index       
    cat_name = class_label[cat_label]    # get a category from category_label using class_label dictionary
    title = 'Reference_image' + ': ' + cat_name
    plt.title(title)
    plt.imshow(img1)
    print('\n')
    
    
    fig = plt.figure(figsize=(10,10))
    #print('Recommended images')
    for j in range(len(list3)):
        i = list3[j]
        categoty_label = df_train["category"][i]       # get a category_label from index       
        category_name = class_label[categoty_label]    # get a category from category_label using class_label dictionary
        img = mpimg.imread(df_train["img_path"][i])    # reading a image using mpimg
        ax = fig.add_subplot(2,3,j+1)
        ax.imshow(img)
        plt.title(category_name)


    return vec3

warnings.filterwarnings("ignore")

In the following cell the input is randomly inputted in a one-hot encoded format.That is if your prefer weather condition is sunny
and you are attending an official event then the sunny and official variables are to be set to 1 while the rest are set to 0.

In [ ]:
# change this cell according to your attribute preferce
sunny = 1
rainy = 0
cold  = 0
normal = 0
party = 0
casual = 0
official = 1

In [ ]:
# converting above values in numpy array
att_vec = np.array([sunny,rainy,cold,normal,party,casual,official], dtype=float)
att_vec